In [4]:
import pandas as pd
import numpy as np
#import time
#from twython import Twython, TwythonError, TwythonRateLimitError
import json
import sklearn
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer #will not need if using nltk for tokens
#import pandas_profiling #may not need
#from pandas_profiling.utils.cache import cache_file #may not need
import seaborn as sns
import nltk
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

In [5]:
df1 = pd.read_csv('data_11_30_20_df.csv', low_memory= False)
#df2 = pd.read_csv('data_12-06-20_df.csv', low_memory= False)
#df3 = pd.read_csv('data_12-13-20_df.csv', low_memory= False)

In [6]:
print(df1.head())
print(df1.info())

                       created_at                   id               id_str  \
0  Mon Nov 30 03:47:59 +0000 2020  1333256437225713664  1333256437225713664   
1  Mon Nov 30 03:39:03 +0000 2020  1333254185278693378  1333254185278693378   
2  Mon Nov 30 03:32:31 +0000 2020  1333252543758086145  1333252543758086145   
3  Mon Nov 30 03:20:08 +0000 2020  1333249426807197702  1333249426807197702   
4  Mon Nov 30 03:20:03 +0000 2020  1333249404262821888  1333249404262821888   

                                                text  truncated  \
0  RT @GlacierArtist: https://t.co/WRdskuo5hr\n"E...      False   
1  RT @CowboyParamedic: HUGE ONE DAY SALE!!!! Nov...      False   
2  Teddy Roosevelt was onto something when select...       True   
3  RT @TrailofHighways: Bridge of timber in a for...      False   
4  Bridge of timber in a forest of timber, \n.\n....       True   

                                              source  in_reply_to_status_id  \
0  <a href="http://twitter.com/download/iph

In [7]:
df_nort = df1[~df1.text.str.contains('RT')] #Removes retweets

df_nort = df_nort.drop_duplicates(subset='text') #Removes duplicate tweets

tweet = df_nort.text #Separates tweets from other twitter data

tweet = tweet.str.lower() #Make lower case

tweet_nopunc = tweet.str.replace('[^!#\w\s]','') #Removes punctuation

tweet_nopunc = tweet.str.replace('\n',' ') #Removes new line character

tweet_nopunc = tweet.str.replace('...', '') #Removes ellipses

tweet_nopunc = tweet.str.replace('!', '') #Removes !

tweet_nopunc = tweet.str.replace(r'http\S+', '') #Removes hyperlinks

In [8]:
#tfidf = TfidfVectorizer() #Create word frequency array

#csr_mat = tfidf.fit_transform(tweet_nopunc)

#print(csr_mat.toarray())

#words = tfidf.get_feature_names() 

#print(words)

In [9]:
tweet2 = tweet_nopunc.str.cat(sep=' ') #Convert series to list

tknzr = TweetTokenizer(strip_handles=True)
tokens = tknzr.tokenize(tweet2)

In [10]:
print(tokens)

['teddy', 'roosevelt', 'was', 'onto', 'something', 'when', 'selecting', 'the', 'standard', 'bearer', 'for', 'his', 'political', 'party', '.', '#bullmoose', '#rmnp', '…', 'bridge', 'of', 'timber', 'in', 'a', 'forest', 'of', 'timber', ',', '.\n.\n.', '#optoutside', '#hikerchat', '#flyfishing', '#livelifeoutside', '#sleepoutside', '…', 'the', 'perfect', 'yosemite', 'shot', '!', 'have', 'you', 'been', 'to', 'yosemite', 'yet', '!', '?', 'if', 'not', ',', 'is', 'it', 'on', 'your', 'bucket', 'list', '?', '___', '…', 'prairie', 'creek', 'redwoods', 'state', 'park', 'today', '.', 'image', 'credit', 'kurt', 'angersbach', '/', 'westernlabs', '.', 'learn', 'more', 'at', '…', 'bucket', 'list', 'alert', '!', 'in', '2', 'days', 'rafting', 'trips', 'to', 'the', '#grandcanyon', 'go', 'on', 'sale', 'for', '2022', '.', 'in', 'years', 'past', ',', 'the', 'trips', 'have', 's', '…', 'next', 'park', 'that', 'astonished', 'me', 'beyond', 'belief', '#findyourpark', '#nps', '@', 'joshua', 'tree', 'national', 'p

In [11]:
stop_words = stopwords.words('english')
tweet_final = list()
for t in tokens:
    if t not in stop_words:
        tweet_final.append(t)
print(tweet_final)

['teddy', 'roosevelt', 'onto', 'something', 'selecting', 'standard', 'bearer', 'political', 'party', '.', '#bullmoose', '#rmnp', '…', 'bridge', 'timber', 'forest', 'timber', ',', '.\n.\n.', '#optoutside', '#hikerchat', '#flyfishing', '#livelifeoutside', '#sleepoutside', '…', 'perfect', 'yosemite', 'shot', '!', 'yosemite', 'yet', '!', '?', ',', 'bucket', 'list', '?', '___', '…', 'prairie', 'creek', 'redwoods', 'state', 'park', 'today', '.', 'image', 'credit', 'kurt', 'angersbach', '/', 'westernlabs', '.', 'learn', '…', 'bucket', 'list', 'alert', '!', '2', 'days', 'rafting', 'trips', '#grandcanyon', 'go', 'sale', '2022', '.', 'years', 'past', ',', 'trips', '…', 'next', 'park', 'astonished', 'beyond', 'belief', '#findyourpark', '#nps', '@', 'joshua', 'tree', 'national', 'park', 'getanalysis', ':', '#congratulations', '#emilyharrington', ',', 'first', 'woman', 'climb', 'rockface', '#elcapitan', 'day', ',', 'taking', 'th', '…', 'thanks', '!', 'total', 'accident', 'really', '.', '’', 'even',

In [12]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /opt/anaconda3/lib/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [13]:
"""def sentiment_analysis(x):
    score = SentimentIntensityAnalyzer().polarity_scores(x)
    print(score)
    return
sentiment_analysis(tweet2)"""

sid = SentimentIntensityAnalyzer()
ss = sid.polarity_scores(tweet2)
print('Initial Sentiment Analysis w/hashtags incldued:',ss)

Initial Sentiment Analysis w/hashtags incldued: {'neg': 0.025, 'neu': 0.782, 'pos': 0.193, 'compound': 1.0}
